In [19]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns


import sklearn

from sklearn.preprocessing import LabelEncoder

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import BaggingClassifier
from sklearn import tree

In [20]:
class LoadData():
    def __init__(self,train_data):
        self.data = pd.read_csv(train_data)        
    def get_data(self):
        self.train,self.test = train_test_split(self.data,test_size = 0.1,random_state = 0,shuffle=False)
        self.train = self.train.dropna(axis=0)
        

In [21]:
train_data = os.path.join('Data','train.csv')
load_data_obj = LoadData(train_data)

In [25]:
load_data_obj.train.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y
5,LP001011,Male,Yes,2,Graduate,Yes,5417,4196.0,267.0,360.0,1.0,Urban,Y


In [23]:
class Preprocessing():
        
    def pre_processing(self,dataset):
        label_encoder = LabelEncoder()
        for dataset in data_cleaner:    
            dataset['Gender'] = label.fit_transform(dataset['Gender'])
            dataset['Married'] = label.fit_transform(dataset['Married'])
            dataset['Education'] = label.fit_transform(dataset['Education'])
            dataset['Self_Employed'] = label.fit_transform(dataset['Self_Employed'])
            dataset['Property_Area'] = label.fit_transform(dataset['Property_Area'])
            dataset['Loan_Status'] = label.fit_transform(dataset['Loan_Status'])
            
        self.x_train = self.train.drop('Loan_Status',axis=1)
        self.x_train = self.train.drop('Loan_ID',axis=1)
        self.y_train = self.train['Loan_Status']
        self.x_test = self.test.drop('Loan_Status',axis=1)
        self.y_test = self.test['Loan_Status']
    
    def data_analysis(self):
        sns.barplot(x = 'Gender', y = 'Loan_Status', data=self.train)
        sns.barplot(x = 'ApplicantIncome', y = 'Loan_Status',data=self.train)
        sns.barplot(x = 'Self_Employed', y = 'Loan_Status', data=self.train)
        #plt.scatter(self.x_train, self.y_train)
        #plt.show()
    def get_preprocess_data(self,dataset):
        dataset = self.pre_processing(dataset)


In [5]:
class GenModel():
    def __init__(self):
        self.model = None
        self.x_train = load_data_obj.x_train
        self.y_train = load_data_obj.y_train
        self.x_test = load_data_obj.x_test
        self.y_test = load_data_obj.y_test
        
    def train(self):
        self.model = BaggingClassifier(tree.DecisionTreeClassifier(random_state=1))
        self.model.fit(self.x_train,self.y_train)
        print(self.model.score(self.x_test,self.y_test))

In [6]:
model_obj = GenModel()
model_obj.train()

ValueError: could not convert string to float: 'LP002308'